In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!rm -rf /content/imagenes
!unzip /content/drive/My\ Drive/Colab\ Notebooks/imagenes -d /content

In [0]:
# Terminar de rellenar este bloque con lo que vaya haciendo falta

# Importar librerías necesarias
import numpy as np
import keras
import keras.utils as np_utils
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Importar el optimizador a usar
from keras.optimizers import SGD

# Importar modelos y capas específicas que se van a usar
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
# Importar el modelo ResNet50 y su respectiva función de preprocesamiento,
# que es necesario pasarle a las imágenes para usar este modelo

from keras.applications.resnet50 import ResNet50, preprocess_input


# Importar el optimizador a usar
from keras.optimizers import SGD


batch_size = 32
epochs = 20
#########################################################################
################## FUNCIÓN PARA LEER LAS IMÁGENES #######################
#########################################################################

# Dado un fichero train.txt o test.txt y el path donde se encuentran los
# ficheros y las imágenes, esta función lee las imágenes
# especificadas en ese fichero y devuelve las imágenes en un vector y
# sus clases en otro.

def leerImagenes(vec_imagenes, path):
  clases = np.array([img.split('/')[0] for img in vec_imagenes])
  imagenes = np.array([img_to_array(load_img(path + "/" + img,
                                             target_size = (224, 224)))
                       for img in vec_imagenes])
  return imagenes, clases

#########################################################################
############# FUNCIÓN PARA CARGAR EL CONJUNTO DE DATOS ##################
#########################################################################

# Usando la función anterior, y dado el path donde se encuentran las
# imágenes y los archivos "train.txt" y "test.txt", devuelve las
# imágenes y las clases de train y test para usarlas con keras
# directamente.

def cargarDatos(path):
  # Cargamos los ficheros
  train_images = np.loadtxt(path + "/train.txt", dtype = str)
  test_images = np.loadtxt(path + "/test.txt", dtype = str)

  # Leemos las imágenes con la función anterior
  train, train_clases = leerImagenes(train_images, path)
  test, test_clases = leerImagenes(test_images, path)

  # Pasamos los vectores de las clases a matrices
  # Para ello, primero pasamos las clases a números enteros
  clases_posibles = np.unique(np.copy(train_clases))
  for i in range(len(clases_posibles)):
    train_clases[train_clases == clases_posibles[i]] = i
    test_clases[test_clases == clases_posibles[i]] = i

  # Después, usamos la función to_categorical()
  train_clases = np_utils.to_categorical(train_clases, 200)
  test_clases = np_utils.to_categorical(test_clases, 200)

  # Barajar los datos
  train_perm = np.random.permutation(len(train))
  train = train[train_perm]
  train_clases = train_clases[train_perm]

  test_perm = np.random.permutation(len(test))
  test = test[test_perm]
  test_clases = test_clases[test_perm]

  return train, train_clases, test, test_clases



#########################################################################
######## FUNCIÓN PARA OBTENER EL ACCURACY DEL CONJUNTO DE TEST ##########
#########################################################################

# Esta función devuelve el accuracy de un modelo, definido como el
# porcentaje de etiquetas bien predichas frente al total de etiquetas.
# Como parámetros es necesario pasarle el vector de etiquetas verdaderas
# y el vector de etiquetas predichas, en el formato de keras (matrices
# donde cada etiqueta ocupa una fila, con un 1 en la posición de la clase
# a la que pertenece y 0 en las demás).

def calcularAccuracy(labels, preds):
  labels = np.argmax(labels, axis = 1)
  preds = np.argmax(preds, axis = 1)

  accuracy = sum(labels == preds)/len(labels)

  return accuracy

#########################################################################
## FUNCIÓN PARA PINTAR LA PÉRDIDA Y EL ACCURACY EN TRAIN Y VALIDACIÓN ###
#########################################################################

# Esta función pinta dos gráficas, una con la evolución de la función
# de pérdida en el conjunto de train y en el de validación, y otra
# con la evolución del accuracy en el conjunto de train y en el de
# validación. Es necesario pasarle como parámetro el historial
# del entrenamiento del modelo (lo que devuelven las funciones
# fit() y fit_generator()).

def mostrarEvolucion(hist):

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training loss', 'Validation loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training accuracy', 'Validation accuracy'])
  plt.show()

"""## Usar ResNet50 preentrenada en ImageNet como un extractor de características"""

# Definir un objeto de la clase ImageDataGenerator para train y otro para test
# con sus respectivos argumentos.


# Definir el modelo ResNet50 (preentrenado en ImageNet y sin la última capa).


# Extraer las características las imágenes con el modelo anterior.


# Las características extraídas en el paso anterior van a ser la entrada
# de un pequeño modelo de dos capas Fully Conected, donde la última será la que
# nos clasifique las clases de Caltech-UCSD (200 clases). De esta forma, es
# como si hubiéramos fijado todos los parámetros de ResNet50 y estuviésemos
# entrenando únicamente las capas añadidas. Definir dicho modelo.
# A completar: definición del modelo, del optimizador y compilación y
# entrenamiento del modelo.
# En la función fit() puedes usar el argumento validation_split



def compile(model):
    optEj3 = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=optEj3,
                  metrics=['accuracy'])

def entrenamiento(model, x_train, y_train, batch_size, epochs):
    # Entrenamos el modelo
    historial = model.fit(x_train, y_train,
             batch_size = batch_size,
             epochs = epochs,
             verbose = 1,
             validation_split = 0.1)

    return historial

def entrenamiento_datagen(model, datagen, x_train, y_train, batch_size, epochs):
    historial = model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size, subset = 'training'),
       epochs = epochs,
       steps_per_epoch = len(x_train)*0.9/batch_size,
       verbose = 1,
       validation_data = datagen.flow(x_train, y_train, batch_size = batch_size, subset = 'validation'),
       validation_steps = len(x_train)*0.1/batch_size)

    return historial

def prediccion(model, datagen, x):
    return model.predict_generator(datagen.flow(x, batch_size = 1, shuffle = False), steps = len(x))

def evaluacion(model, x_test, y_test):
    return model.evaluate(x_test, y_test)

def evaluacion_datagen(model, datagen, x_test, y_test):
    return model.evaluate_generator(datagen.flow(x_test, y_test, batch_size = 1, shuffle = False), steps = len(x_test))


def definicionModelo():
    model = Sequential()
    model.add(Dense(1024, activation = 'relu', input_shape = (2048, )))
    model.add(Dense(200, activation = 'softmax'))
    return model


def ej3_1(batch_size, epochs):
    print("\n\nCargando imágenes...")
    x_train, y_train, x_test, y_test = cargarDatos("./imagenes")
    print("\nTodo OK.")

    datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input)
    datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)

    resnet50 = ResNet50(weights='imagenet', include_top = False, pooling = "avg", input_shape = (224, 224, 3))

    print("\nExtraigo características...")
    features_train = prediccion(resnet50, datagen_train, x_train)
    features_test = prediccion(resnet50, datagen_test, x_test)
    print("\nCaracterísticas extraídas.")
    model = definicionModelo()
    compile(model)
    print("\nEntrenando...")
    historial = entrenamiento(model, x_train, y_train, batch_size, epochs)
    print("\nFin de entrenamiento, evaluando...")
    score = evaluacion(model, x_test, y_test)
    mostrarEvolucion(historial)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])


ej3_1(batch_size, epochs)
input("Fin del apartado 1, pulsa Enter")

In [0]:
"""## Reentrenar ResNet50 (fine tunning)"""

def definicionModeloApartado2(x):
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.7)(x)
    last = Dense(200, activation='softmax')(x)
    return last

def ej3_2(batch_size, epochs):
    print("\n\nCargando imágenes...")
    x_train, y_train, x_test, y_test = cargarDatos("./imagenes")
    print("\nTodo OK.")

    datagen_train = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split = 0.1)
    datagen_test = ImageDataGenerator(preprocessing_function = preprocess_input)

    resnet50 = ResNet50(weights='imagenet', include_top = False, pooling = "avg", input_shape = (224, 224, 3))

    last = definicionModeloApartado2(resnet50.output)
    new_model = Model(inputs = resnet50.input, outputs = last)

    print("\nCompilando el modelo con el optimizador...")
    compile(new_model)
    print("\nEntrenando...")
    historial = entrenamiento_datagen(new_model, datagen_train, x_train, y_train, batch_size, epochs)
    print("\nFin de entrenamiento, evaluando...")
    score = evaluacion_datagen(new_model, datagen_test, x_test, y_test)

    mostrarEvolucion(historial)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])


ej3_2(batch_size, epochs)
input("Fin del apartado 2, pulsa Enter")